In [1]:
import pymysql

In [2]:
db = pymysql.connect("***","***","***","***",charset='utf8')

In [3]:
cursor = db.cursor()

In [4]:
sql = "SELECT * FROM RAW_DATA_train"

In [5]:
try:
   # 执行SQL语句
   cursor.execute(sql)
   # 获取所有记录列表
   results = cursor.fetchall()
except:
   print ("Error: unable to fetch data")

In [6]:
import numpy as np

In [7]:
martir=[]
for row in results:
    #print(type(row[1]))
    label=row[1]
    text=row[2]
    pair=[]
    pair.append(text)
    pair.append(label)
    martir.append(pair)

In [8]:
mat_array=np.array(martir)

In [9]:
# DIS_SQL="SELECT DISTINCT label FROM RAW_DATA_train"
# try:
#    # 执行SQL语句
#    cursor.execute(DIS_SQL)
#    # 获取所有记录列表
#    label_results = cursor.fetchall()
# except:
#    print ("Error: unable to fetch data")
 
# # 关闭数据库连接
# db.close()

In [10]:
# mapping_dict={'exterior':'1','driving':'2','safety':'3','performance':'4','space':'5','interior':'6','fuel economy':'7'}

In [11]:
# for i in mat_array:
#     i[0]=mapping_dict[i[0]]

In [12]:
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [13]:
df=pd.DataFrame(mat_array)

In [14]:
df=df.sample(frac=1).reset_index(drop=True)

In [15]:
df.columns = ['text','label']

In [16]:
save_df=df.head(4)

In [17]:
save_df.to_csv('st.tsv',sep='\t',index=False)

In [18]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/I348655/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

In [21]:
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) 

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 
df['text'] = detokenized_doc

In [22]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, test_trn = train_test_split(df, stratify = df['label'], test_size = 0.3, random_state = 12)

In [23]:
df_trn.head(5)

,text,label
42725,In addition proficiency cargo hauling E class ...,space
32841,The general design regardless body style excep...,driving
40173,EPA rated fuel economy automatic equipped four...,fuel economy
5114,You might assume CLA sedan also small trunk re...,space
19195,Read rate cars The powertrain available QX wel...,driving


In [24]:
df_trn.shape

(70900, 2)

In [25]:
df_t=df_trn[0:39630]

In [26]:
df_v=df_trn[39631:70900]

In [27]:
df_t.head(5)

,text,label
42725,In addition proficiency cargo hauling E class ...,space
32841,The general design regardless body style excep...,driving
40173,EPA rated fuel economy automatic equipped four...,fuel economy
5114,You might assume CLA sedan also small trunk re...,space
19195,Read rate cars The powertrain available QX wel...,driving


In [28]:
df_v.head(10)

,text,label
94225,The big flat kilowatt hour battery pack floor ...,fuel economy
43991,In side impact testing Impala equipped side ai...,safety
28543,Energy comes courtesy lithium ion battery pack...,fuel economy
85694,The Laramie Limited trim adds monotone paint c...,exterior
42785,The Frontier crew cab received top rating Good...,safety
17827,It looks like one solid piece viewed side Car ...,exterior
78550,That frugal Ram turbodiesel top mpg heavier tr...,fuel economy
14317,Technically four cylinder models handle bit be...,driving
86802,Just years ago Acura ILX range would near lead...,fuel economy
47704,The communicative steering enhances confidence...,driving


In [29]:
test_trn.head(5)

,text,label
17120,The CVT lets engine slur higher rpm slightest ...,performance
38940,Optional equipment includes inflatable rear se...,safety
24544,While cargo carrying numbers average storage s...,space
39090,The Limited version gets upgraded leather upho...,interior
1375,Because battery pack mounted beneath floor Bol...,fuel economy


In [30]:
df_t.to_csv('DATA_DIR/train.tsv',sep='\t',index=False)

In [31]:
test_trn.to_csv('DATA_DIR/test.tsv',sep='\t',index=False)

In [32]:
df_v.to_csv('DATA_DIR/dev.tsv',sep='\t',index=False)